# Calculate Screen time of an actor:

In [1]:
# Imports

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt    
%matplotlib inline

import cv2  
import math   
from keras.preprocessing import image   # for preprocessing the images
from keras.utils import np_utils
from skimage.transform import resize  

Using TensorFlow backend.


In [3]:
count = 0

videoFile = "Tom and jerry.mp4"
cap = cv2.VideoCapture(videoFile)    # Capturing Video
frameRate = cap.get(5)
x=1

while(cap.isOpened()):
    frameId = cap.get(1)  #current frame number
    ret, frame = cap.read()
    if (ret != True):
        break
    if (frameId % math.floor(frameRate) == 0):
        filename ="frame%d.jpg" % count;
        count+=1
        cv2.imwrite(filename, frame)
cap.release()
print ("Frame Capturing done!")

Frame Capturing done!


In [4]:
frameRate

30.0

In [5]:
data = pd.read_csv('mapping.csv')   
data.head() 

,Image_ID,Class
0,frame0.jpg,1
1,frame1.jpg,1
2,frame2.jpg,2
3,frame3.jpg,2
4,frame4.jpg,2


In [7]:
X = [ ] 

X = [ ]     
for img_name in data.Image_ID:
    img = plt.imread('' + img_name)
    X.append(img)  
X = np.array(X)

In [10]:
X.shape

(298, 720, 1280, 3)

In [11]:
y = data.Class
dummy_y = np_utils.to_categorical(y)  #converting to categorical variables

In [12]:
dummy_y

array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0

### VGG-16 Pre-trained model:
    VGG16 is a convolutional neural network model proposed by K. Simonyan and A. Zisserman from the University of Oxford in the paper “Very Deep Convolutional Networks for Large-Scale Image Recognition”. The model achieves 92.7% top-5 test accuracy in ImageNet.
    
Takes an input image of shape (224 X 224 X 3)

#### Re-sizing image using skimage.transform

In [13]:
image = []
for i in range(0,X.shape[0]):
    a = resize(X[i], preserve_range=True, output_shape=(224,224)).astype(int)      # reshaping to 224*224*3
    image.append(a)
X = np.array(image)

#### Pre-processing as per model requirement:

In [14]:
from keras.applications.vgg16 import preprocess_input
X = preprocess_input(X, mode='tf')  

In [29]:
# preparing the validation set

from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, dummy_y, test_size=0.3, random_state=42) 

## Building model:

In [30]:
# Imports

from keras.models import Sequential
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, InputLayer, Dropout

In [31]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))    
# include_top=False to remove the top layer

In [32]:
X_train = base_model.predict(X_train)
X_valid = base_model.predict(X_valid)

In [19]:
X_train.shape, X_valid.shape

((208, 7, 7, 512), (90, 7, 7, 512))

### Re-shaping to 1D

In [33]:
X_train = X_train.reshape(208, 7*7*512)      # converting to 1-D
X_valid = X_valid.reshape(90, 7*7*512)

##### Making images zero-centered (  dataset is symmetric,which makes it easier to converge)

In [35]:
train = X_train/X_train.max()      # centering the data
X_valid = X_valid/X_train.max()

In [36]:
# i. Building the model
model = Sequential()
model.add(InputLayer((7*7*512,)))    # input layer
model.add(Dense(units=1024, activation='sigmoid')) # hidden layer
model.add(Dense(3, activation='softmax'))    # output layer ( 3 neurons as we have 3 classes to predict)

In [37]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 1024)              25691136  
_________________________________________________________________
dense_6 (Dense)              (None, 3)                 3075      
Total params: 25,694,211
Trainable params: 25,694,211
Non-trainable params: 0
_________________________________________________________________


In [38]:
#Compiling the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [39]:
#Training the model

model.fit(train, y_train, epochs=100, validation_data=(X_valid, y_valid))

Train on 208 samples, validate on 90 samples
Epoch 1/100
208/208 [==============================] - 4s 20ms/step - loss: 1.1597 - accuracy: 0.5288 - val_loss: 1.0599 - val_accuracy: 0.3889
Epoch 2/100
208/208 [==============================] - 2s 11ms/step - loss: 0.4644 - accuracy: 0.7885 - val_loss: 1.1116 - val_accuracy: 0.3889
Epoch 3/100
208/208 [==============================] - 2s 11ms/step - loss: 0.3421 - accuracy: 0.8990 - val_loss: 0.9935 - val_accuracy: 0.4333
Epoch 4/100
208/208 [==============================] - 2s 11ms/step - loss: 0.1810 - accuracy: 0.9615 - val_loss: 0.9545 - val_accuracy: 0.5222
Epoch 5/100
208/208 [==============================] - 2s 11ms/step - loss: 0.0944 - accuracy: 0.9760 - val_loss: 0.9474 - val_accuracy: 0.3889
Epoch 6/100
208/208 [==============================] - 2s 11ms/step - loss: 0.0574 - accuracy: 0.9904 - val_loss: 0.9325 - val_accuracy: 0.5667
Epoch 7/100
208/208 [==============================] - 2s 11ms/step - loss: 0.0460 - accura

208/208 [==============================] - 2s 11ms/step - loss: 0.0228 - accuracy: 0.9952 - val_loss: 0.8182 - val_accuracy: 0.6889
Epoch 58/100
208/208 [==============================] - 2s 11ms/step - loss: 0.0203 - accuracy: 0.9952 - val_loss: 0.8366 - val_accuracy: 0.6111
Epoch 59/100
208/208 [==============================] - 2s 11ms/step - loss: 0.0074 - accuracy: 0.9952 - val_loss: 0.8678 - val_accuracy: 0.5111
Epoch 60/100
208/208 [==============================] - 2s 11ms/step - loss: 0.0128 - accuracy: 0.9952 - val_loss: 0.8552 - val_accuracy: 0.5667
Epoch 61/100
208/208 [==============================] - 2s 10ms/step - loss: 0.0093 - accuracy: 0.9952 - val_loss: 0.8251 - val_accuracy: 0.6222
Epoch 62/100
208/208 [==============================] - 2s 11ms/step - loss: 0.0113 - accuracy: 0.9952 - val_loss: 0.8246 - val_accuracy: 0.6222
Epoch 63/100
208/208 [==============================] - 2s 12ms/step - loss: 0.0141 - accuracy: 0.9904 - val_loss: 0.8327 - val_accuracy: 0.611

In [40]:
count = 0
videoFile = "Tom and Jerry 3.mp4"
cap = cv2.VideoCapture(videoFile)
frameRate = cap.get(5) #frame rate
x=1
while(cap.isOpened()):
    frameId = cap.get(1) #current frame number
    ret, frame = cap.read()
    if (ret != True):
        break
    if (frameId % math.floor(frameRate) == 0):
        filename ="test%d.jpg" % count;count+=1
        cv2.imwrite(filename, frame)
cap.release()
print ("Done!")

Done!


In [41]:
test = pd.read_csv('test.csv')

In [42]:
test_image = []
for img_name in test.Image_ID:
    img = plt.imread('' + img_name)
    test_image.append(img)
test_img = np.array(test_image)

In [44]:
test_image = []
for i in range(0,test_img.shape[0]):
    a = resize(test_img[i], preserve_range=True, output_shape=(224,224)).astype(int)
    test_image.append(a)
test_image = np.array(test_image)

In [46]:
# preprocessing the images
test_image = preprocess_input(test_image, mode='tf')

# extracting features from the images using pretrained model
test_image = base_model.predict(test_image)

# converting the images to 1-D form
test_image = test_image.reshape(186, 7*7*512)

# zero centered images
test_image = test_image/test_image.max()

In [47]:
predictions = model.predict_classes(test_image)

In [48]:
print("The screen time of JERRY is", predictions[predictions==1].shape[0], "seconds")
print("The screen time of TOM is", predictions[predictions==2].shape[0], "seconds")

The screen time of JERRY is 5 seconds
The screen time of TOM is 129 seconds
